In [28]:
from scripts.rag.hybridrag import hybrid_rag_sc
from scripts.rag.naiverag import naive_rag_sc
from scripts.rag.pretty_print import pretty_print_rag_answer

In [19]:
query = "Who uses ChatGPT more, men or women, and how does this change by 2025"
hybrid_answer = hybrid_rag_sc.run(query=query)

Batches: 100%|██████████| 1/1 [00:00<00:00, 11.69it/s]


In [29]:
# Test the pretty print functions
pretty_print_rag_answer(hybrid_answer, "Hybrid RAG", query)

🔍 HYBRID RAG ANSWER
📝 Query: Who uses ChatGPT more, men or women, and how does this change by 2025
--------------------------------------------------------------------------------
💬 Answer:
   Initially, a significant share (around 80%) of weekly active users of
   ChatGPT in the first few months after its release were individuals with
   typically masculine first names. However, by the first half of 2025,
   this gap began to close, and the share of active users with typically
   feminine names reached near parity with those having typically masculine
   names. By June 2025, active users were more likely to have typically
   feminine names, indicating that gender gaps in ChatGPT usage had closed
   substantially over time.

📚 Source Documents (3 found):
--------------------------------------------------
1. Source: Unknown source
   Preview: The prompts for each of these automated classifiers (with the
exception of interaction quality) are available in Appendix A. Values represent the 

In [26]:
naive_answer = naive_rag_sc.run(query=query)

Batches: 100%|██████████| 1/1 [00:00<00:00,  4.86it/s]



In [30]:
pretty_print_rag_answer(naive_answer, "Naive RAG", query)

🔍 NAIVE RAG ANSWER
📝 Query: Who uses ChatGPT more, men or women, and how does this change by 2025
--------------------------------------------------------------------------------
💬 Answer:
   Based on the provided information, initially, around 80% of the weekly
   active users (WAU) in the first few months after ChatGPT was released
   were by users with typically masculine first names. However, by the
   first half of 2025, the share of active users with typically feminine
   and typically masculine names reached near-parity. By June 2025, it was
   observed that active users were more likely to have typically feminine
   names, suggesting that gender gaps in ChatGPT usage have closed
   substantially over time.



Batches: 100%|██████████| 1/1 [00:00<00:00,  4.72it/s]


In [23]:
naive_answer

{'replies': ['Initially, a significant share (around 80%) of the weekly active users of ChatGPT in the first few months after its release were by users with typically masculine first names. However, by the first half of 2025, the share of active users with typically feminine and typically masculine names reached near-parity. By June 2025, active users were more likely to have typically feminine names, suggesting that the gender gaps in ChatGPT usage have closed substantially over time.']}